In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision import models
from torchvision.models.resnet import resnet34
from torchvision.transforms.transforms import Resize
import torch.nn.functional as F
from PIL import Image

if torch.cuda.is_available():
        device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

transform = transforms.Compose([
transforms.Resize(224),
transforms.ToTensor(),
transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])
#采用自带的Cifar100
trainset = torchvision.datasets.CIFAR100(root='./data_CIFAR100', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
 
testset = torchvision.datasets.CIFAR100(root='./data_CIFAR100', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)



Files already downloaded and verified
Files already downloaded and verified


In [2]:
student_model=models.mobilenet_v2(pretrained=False)

student_model.classifier = nn.Sequential( 
    #重新定义特征层，根据需要可以添加自己想要的Linear层
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(in_features=1280, out_features=100),  #多加几层都没关系
    #nn.LogSoftmax(dim=1)
)
 

/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
import pytorch_influence_functions as ptif
import numpy as np
def get_default_config():
    """Returns a default config file"""
    config = {
        'outdir': './',
        'seed': 42,
        'gpu': -1,
        'dataset': 'CIFAR100',
        'num_classes': 5,
        'test_sample_num': 1,
        'test_start_index': 0, #change
        'recursion_depth': 1,
        'r_averaging': 1,
        'scale': None,
        'damp': None,
        'calc_method': 'img_wise',
        'log_filename': None,
    }

    return config

In [ ]:
config = get_default_config()
student_save_path='./Dis_resnet50(T)_mobilebetv2(S)_cifar100_epoch10_withif_1.pkl'
student_model.load_state_dict(torch.load(student_save_path))
#ptif.init_logging(config['outdir'])
influences = ptif.calc_img_wise(config, student_model, trainloader, testloader)

Calc. s_test recursions: [=================================================] 1 / 1
Averaging r-times: [=======================================================] 1 / 1
